In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize

In [21]:
df = pd.read_csv('SpotifyFeatures.csv')  # load dataset
df

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.24600,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.95200,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.70300,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.95000,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,Soul,Slave,Son Of Slide,2XGLdVl7lGeq8ksM6Al7jT,39,0.00384,0.687,326240,0.714,0.544000,D,0.0845,-10.626,Major,0.0316,115.542,4/4,0.962
232721,Soul,Jr Thomas & The Volcanos,Burning Fire,1qWZdkBl4UVPj9lK6HuuFM,38,0.03290,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,0.0337,113.830,4/4,0.969
232722,Soul,Muddy Waters,(I'm Your) Hoochie Coochie Man,2ziWXUmQLrXTiYjCg2fZ2t,47,0.90100,0.517,166960,0.419,0.000000,D,0.0945,-8.282,Major,0.1480,84.135,4/4,0.813
232723,Soul,R.LUM.R,With My Words,6EFsue2YbIG4Qkq8Zr9Rir,44,0.26200,0.745,222442,0.704,0.000000,A,0.3330,-7.137,Major,0.1460,100.031,4/4,0.489


In [22]:
# pick audio features

# acousticness, danceability, energy, instrumentalness, liveness, loudness, speechinevalence,

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
genre_encoded = encoder.fit_transform(df[['genre']])

feat_cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness','valence']

In [ ]:
feature_weights = np.array([1.0, 1.5, 1.5, 0.8, 1.5, 0.7, 1.2, 1.3])
genre_weight = 2.0  # genre importance
X = StandardScaler().fit_transform(df[feat_cols])
X_combined = np.hstack((X, genre_encoded))
# Extend weight vector to match combined shape
feature_weights = np.concatenate([
    feature_weights,
    np.full(genre_encoded.shape[1], genre_weight)
])

# Apply weights
X_weighted = X_combined * feature_weights

X_weighted = normalize(X_weighted)

In [49]:
X_weighted

array([[ 0.17443475, -0.34112242,  0.49273321, ...,  0.        ,
         0.        ,  0.        ],
       [-0.11509607,  0.09594711,  0.3149614 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.41958553,  0.22399332, -0.63909278, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.44578483, -0.08969171, -0.25698394, ...,  0.59405756,
         0.        ,  0.        ],
       [-0.10538101,  0.54051614,  0.26575687, ...,  0.7016812 ,
         0.        ,  0.        ],
       [-0.25146077,  0.54122251, -0.18903903, ...,  0.65774464,
         0.        ,  0.        ]])

In [52]:
def compute_similarity_batched(features, batch_size=1000):
    n_samples = features.shape[0]
    similarities = []
    # Only compute what you need, when you need it
    for i in range(0, n_samples, batch_size):
        batch = features[i:i+batch_size]
        batch_sim = cosine_similarity(batch, features)
        similarities.append(batch_sim)

    return np.vstack(similarities)

#sim = compute_similarity_batched(X)
#sim

# Top K songs for query song
def find_top_similar_songs(query_song_index, features_matrix, song_names, artist_names, k=5):
  query_features = features_matrix[query_song_index].reshape(1, -1)
  similarities = cosine_similarity(query_features, features_matrix).flatten()

  # Exclude the query song itself
  similarities[query_song_index] = -1

  # Sort all songs by similarity (descending)
  sorted_indices = np.argsort(similarities)[::-1]

  query_name = song_names.iloc[query_song_index]
  seen_names = set([query_name])  # track seen song names
  results = []

  for idx in sorted_indices:
      name = song_names.iloc[idx]
      artist = artist_names.iloc[idx]
      if name not in seen_names:  # ensures no duplicates
          results.append((name, artist, similarities[idx]))
          seen_names.add(name)
      if len(results) == k:
          break

  return results
  '''
  query_features = features_matrix[query_song_index].reshape(1, -1)
  similarities = cosine_similarity(query_features, features_matrix).flatten()

  # Exclude the query song itself
  similarities[query_song_index] = -1

  # Sort all songs by similarity (descending)
  sorted_indices = np.argsort(similarities)[::-1]

  query_name = song_names.iloc[query_song_index]
  results = []

  # Iterate through sorted indices until we collect k unique songs
  for idx in sorted_indices:
      name = song_names.iloc[idx]
      if name != query_name:  # skip same-named songs
          results.append((name, similarities[idx]))
      if len(results) == k:
          break

  return results
  '''
  '''
  query_features = features_matrix[query_song_index].reshape(1, -1)
  similarities = cosine_similarity(query_features, features_matrix).flatten()

  # Exclude the query song itself (same index)
  similarities[query_song_index] = -1

  # Get top K most similar songs
  top_indices = np.argsort(similarities)[::-1][:k]

  # Optionally, also ensure you don’t return songs with identical names
  query_name = song_names.iloc[query_song_index]
  results = []
  for idx in top_indices:
      if song_names.iloc[idx] != query_name:
          results.append((song_names.iloc[idx], similarities[idx]))
      if len(results) == k:
          break

  return results
  '''
''' # Get the feature vector for your query songs
query_features = features_matrix[query_song_index].reshape(1, -1)

# compute similarity only between query song and all others
similarities = cosine_similarity(query_features, features_matrix).flatten()

# get indices of top k+1 songs (including the query song itself)
top_indices = np.argsort(similarities)[::-1][:k+1]

# remove the query song itself from results
top_indices = [idx for idx in top_indices if idx != query_song_index][:k]

# return results
return [(song_names[idx], similarities[idx]) for idx in top_indices]'''



song_name_to_find = "A Lonely Night"
index = df[df['track_name'] == song_name_to_find].index[0]

print("Top 5 songs similar to this song: " + df["track_name"][index])
find_top_similar_songs(index, X_weighted, df["track_name"], df["artist_name"], 5)

Top 5 songs similar to this song: A Lonely Night


[('Uh Huh', 'B2K', np.float64(0.9958074041192481)),
 ('Sugar Daddy', 'Macy Gray', np.float64(0.9953944091785825)),
 ('No More', 'Ruff Endz', np.float64(0.9947985954286201)),
 ('No Strings (feat. Starrah)', 'Kid Ink', np.float64(0.9937255784964086)),
 ('Move Ya Body', 'Nina Sky', np.float64(0.9937133083487046))]

In [27]:
df['genre'].unique()

array(['Movie', 'R&B', 'A Capella', 'Alternative', 'Country', 'Dance',
       'Electronic', 'Anime', 'Folk', 'Blues', 'Opera', 'Hip-Hop',
       "Children's Music", 'Children’s Music', 'Rap', 'Indie',
       'Classical', 'Pop', 'Reggae', 'Reggaeton', 'Jazz', 'Rock', 'Ska',
       'Comedy', 'Soul', 'Soundtrack', 'World'], dtype=object)